In [ ]:
!pip install wikipedia

# The NLTK stopwords had a limited number of stopwords so I used SpaCy for better results
!pip install spacy
!python3 -m spacy download en_core_web_sm

!pip uninstall nltk -y
!pip install nltk

from IPython.display import clear_output
clear_output(wait=True)
print("Wikipedia, SpaCy installation Done")

In [ ]:
import wikipedia
import re
import spacy
sp = spacy.load('en_core_web_sm')

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer, SnowballStemmer, PorterStemmer

lancaster = LancasterStemmer()
snowball = SnowballStemmer(language = 'english')
ps = PorterStemmer()

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Getting names of all places in the world so that we can remove them from the final corpus of the wikipedia pages

!pip install geosky
from geosky import geo_plug

import json
city_names = json.loads(geo_plug.all_State_CityNames())

geography = []
final_places_country_list = []
for i in geo_plug.all_CountryNames():
    final_places_country_list.append(i.lower())
    
for i in city_names:
    for j in (list(i.keys())):
        final_places_country_list.append(j.lower())
    for j in list(i.values()):
        for k in j:
            final_places_country_list.append(k.lower())
            
clear_output(wait = False)

In [ ]:
# Initializing stopwords
all_stopwords = sp.Defaults.stop_words

In [ ]:
def wikipedia_page_content(title_of_page):
    p = wikipedia.page(title = title_of_page, auto_suggest = False)
    return p.content # Content of page.

def get_lowercase_corpus(content):
    corpus = content.split('\n')
    corpus_lower = []
    for i in corpus:
        corpus_lower.append(re.sub(r'[^a-z0-9\-\ ]', '', i.lower()))
        
    return corpus_lower

def get_main_corpus_removing_stopwords(corpus_lower):
    main_corpus = []
    for i in corpus_lower:
        if(len(i) > 10):
            stri = ''
            for j in i.split():
                if (j not in all_stopwords and len(j) > 0 and j not in final_places_country_list):
                    stri += (j + ' ')
            main_corpus.append(stri.strip())
            
    return main_corpus

In [ ]:
def get_filter_words(main_corpus):
    tfidf = TfidfVectorizer()
    tfidf.fit_transform(main_corpus)
    # get idf values
    res = []
    for ele1, ele2 in zip(tfidf.get_feature_names(), tfidf.idf_):
        if(ele1.isnumeric() == False):
            res.append([ele1, ele2])
    return [word[0] for word in sorted(res,key=lambda l:l[1], reverse = False) if len(word[0]) > 4][:20]

In [ ]:
def get_key_words_from_wikipedia_page(page_title):
    content = wikipedia_page_content(page_title)
    corpus_lower = get_lowercase_corpus(content)
    main_corpus = get_main_corpus_removing_stopwords(corpus_lower)
    return list(set(get_filter_words(main_corpus)))

In [ ]:
clear_output(wait = True)
print("Loaded Wikipedia Word Extractor file!")

In [ ]:
# pages = ['K-12', 'K-12_education_in_the_United_States','No_Child_Left_Behind_Act']

# keywords = []
# for i in pages:
#     words = get_key_words_from_wikipedia_page(i)
#     for j in words:
#         keywords.append(j)
        
# print(list(set(keywords)))

In [ ]:
# print(lancaster.stem('schools'))
# print(lancaster.stem('school'))
# print(lancaster.stem('american'))

In [ ]:
# print(snowball.stem('schools'))
# print(snowball.stem('school'))
# print(snowball.stem('secondary'))

In [ ]:
# print(ps.stem('schools'))
# print(ps.stem('school'))
# print(ps.stem('american'))